In [1]:
import pandas as pd


In [2]:
data=pd.read_parquet('..\..\curated\df_ordenes_compras.parquet')

print(data.isnull().sum())

Companía                 0
OrdenCompra              0
Item_Commodity           0
Descripcion              0
Descripcion_items        0
CantidadPedida           0
CantidadRecibida         0
PrecioUnitario           0
Total                    0
NEntregas                0
EstadoLínea              0
CentroCosto              0
EstadoOrdenCompra        0
Clasificacion            0
Proveedor                0
FechaPreparación         0
FechaEntrega            36
AlmacenIngreso       20197
FormadePago            109
Numero                 109
dtype: int64


In [50]:
## datos los nulos , requerimos quitar los campos  FechaEntrega     AlmacenIngreso   Numero    
data_frame_1 = data.drop(columns=['FechaEntrega', 'AlmacenIngreso', 'Numero'])
data_frame_1=data_frame_1[data_frame_1['FormadePago'].notna()]


In [54]:
data_frame_1

,Companía,OrdenCompra,Item_Commodity,Descripcion,Descripcion_items,CantidadPedida,CantidadRecibida,PrecioUnitario,Total,NEntregas,EstadoLínea,CentroCosto,EstadoOrdenCompra,Clasificacion,Proveedor,FechaPreparación,FormadePago
0,1000000,5,11006,CAMPANA EXTRACTORA,I- PATRIMUL COMPUESTO Aer.Susp.x 200 DOSIS ...,1.0,0.0,19576.270000,19576.27,0,Anulada,10305,Anulada,LOC,INSTALACIONES TRC SAC,2018-05-03 16:05:52.973,L/60 Días
1,1000000,1580,11006,CAMPANA EXTRACTORA,I- PATRIMUL COMPUESTO Aer.Susp.x 200 DOSIS ...,1.0,1.0,19576.270000,19576.27,0,Completa,10305,Completada,LOC,INSTALACIONES TRC SAC,2018-07-04 18:28:46.013,Crédito
2,1000000,20494,11006,I- PATRIMUL 20 MCG Aer.Susp.x 200 DOSIS (MEGA...,I- PATRIMUL COMPUESTO Aer.Susp.x 200 DOSIS ...,20.0,20.0,70.200000,1404.00,0,Completa,10808,Completada,Compra Medicinas/M.Medicos,ALFARO S.A.C.,2020-12-03 13:07:45.984,L/120 Días
3,1000000,6,8010,ARCO EN C CIOS ALPHA,CORTIPREX 5MG/5ML SUSP. X 100 ML,1.0,1.0,164749.423730,164749.42,0,Completa,30307,Completada,LOC,SIEMENS HEALTHCARE S.A.C,2018-05-03 17:25:35.040,10% Adelantado - Saldo Crédito (12)
4,1000000,88,8010,CORTIPREX JBE,CORTIPREX 5MG/5ML SUSP. X 100 ML,2.0,2.0,15.439970,30.88,0,Completa,10304,Completada,LOC,ALFARO S.A.C.,2018-05-08 08:30:11.002,L/90 Días
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44073,1000000,37290,7741,SULFADIAZINA 400 G,SULFADIAZINA 400 G,5.0,5.0,67.627119,338.14,0,Completa,10808,Completada,Compra Medicinas/M.Medicos,IMPROVENG S.R.L.,2024-03-19 14:17:40.963,L/120 Días
44074,1000000,37498,7633,TROPICAMIDA 1% X 15 ML,TROPICAMIDA 1% X 15 ML,15.0,15.0,35.644068,534.66,0,Completa,10808,Aprobada,Compra Medicinas/M.Medicos,DIMEXA,2024-04-05 15:51:04.003,L/90 Días
44075,1000000,37498,13580,FENILEFRINA 10% X 5 ML,FENILEFRINA 10% X 5 ML,15.0,15.0,40.347458,605.21,0,Completa,10808,Aprobada,Compra Medicinas/M.Medicos,DIMEXA,2024-04-05 15:51:04.003,L/90 Días
44076,1000000,37499,13582,VABYSMO 120 MG,VABYSMO 120 MG,1.0,1.0,2719.161017,2719.16,0,Completa,10808,Completada,Compra Medicinas/M.Medicos,QUIMICA SUIZA S.A.,2024-04-05 16:06:08.006,Contado


In [55]:
#### 
data_frame_1['FechaPreparación'] = pd.to_datetime(data_frame_1['FechaPreparación'])


In [56]:
data_frame_1

,Companía,OrdenCompra,Item_Commodity,Descripcion,Descripcion_items,CantidadPedida,CantidadRecibida,PrecioUnitario,Total,NEntregas,EstadoLínea,CentroCosto,EstadoOrdenCompra,Clasificacion,Proveedor,FechaPreparación,FormadePago
0,1000000,5,11006,CAMPANA EXTRACTORA,I- PATRIMUL COMPUESTO Aer.Susp.x 200 DOSIS ...,1.0,0.0,19576.270000,19576.27,0,Anulada,10305,Anulada,LOC,INSTALACIONES TRC SAC,2018-05-03 16:05:52.973,L/60 Días
1,1000000,1580,11006,CAMPANA EXTRACTORA,I- PATRIMUL COMPUESTO Aer.Susp.x 200 DOSIS ...,1.0,1.0,19576.270000,19576.27,0,Completa,10305,Completada,LOC,INSTALACIONES TRC SAC,2018-07-04 18:28:46.013,Crédito
2,1000000,20494,11006,I- PATRIMUL 20 MCG Aer.Susp.x 200 DOSIS (MEGA...,I- PATRIMUL COMPUESTO Aer.Susp.x 200 DOSIS ...,20.0,20.0,70.200000,1404.00,0,Completa,10808,Completada,Compra Medicinas/M.Medicos,ALFARO S.A.C.,2020-12-03 13:07:45.984,L/120 Días
3,1000000,6,8010,ARCO EN C CIOS ALPHA,CORTIPREX 5MG/5ML SUSP. X 100 ML,1.0,1.0,164749.423730,164749.42,0,Completa,30307,Completada,LOC,SIEMENS HEALTHCARE S.A.C,2018-05-03 17:25:35.040,10% Adelantado - Saldo Crédito (12)
4,1000000,88,8010,CORTIPREX JBE,CORTIPREX 5MG/5ML SUSP. X 100 ML,2.0,2.0,15.439970,30.88,0,Completa,10304,Completada,LOC,ALFARO S.A.C.,2018-05-08 08:30:11.002,L/90 Días
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44073,1000000,37290,7741,SULFADIAZINA 400 G,SULFADIAZINA 400 G,5.0,5.0,67.627119,338.14,0,Completa,10808,Completada,Compra Medicinas/M.Medicos,IMPROVENG S.R.L.,2024-03-19 14:17:40.963,L/120 Días
44074,1000000,37498,7633,TROPICAMIDA 1% X 15 ML,TROPICAMIDA 1% X 15 ML,15.0,15.0,35.644068,534.66,0,Completa,10808,Aprobada,Compra Medicinas/M.Medicos,DIMEXA,2024-04-05 15:51:04.003,L/90 Días
44075,1000000,37498,13580,FENILEFRINA 10% X 5 ML,FENILEFRINA 10% X 5 ML,15.0,15.0,40.347458,605.21,0,Completa,10808,Aprobada,Compra Medicinas/M.Medicos,DIMEXA,2024-04-05 15:51:04.003,L/90 Días
44076,1000000,37499,13582,VABYSMO 120 MG,VABYSMO 120 MG,1.0,1.0,2719.161017,2719.16,0,Completa,10808,Completada,Compra Medicinas/M.Medicos,QUIMICA SUIZA S.A.,2024-04-05 16:06:08.006,Contado


In [57]:
data_frame_1['Mes'] = data_frame_1['FechaPreparación'].dt.to_period('M')


In [58]:
data_frame_1

,Companía,OrdenCompra,Item_Commodity,Descripcion,Descripcion_items,CantidadPedida,CantidadRecibida,PrecioUnitario,Total,NEntregas,EstadoLínea,CentroCosto,EstadoOrdenCompra,Clasificacion,Proveedor,FechaPreparación,FormadePago,Mes
0,1000000,5,11006,CAMPANA EXTRACTORA,I- PATRIMUL COMPUESTO Aer.Susp.x 200 DOSIS ...,1.0,0.0,19576.270000,19576.27,0,Anulada,10305,Anulada,LOC,INSTALACIONES TRC SAC,2018-05-03 16:05:52.973,L/60 Días,2018-05
1,1000000,1580,11006,CAMPANA EXTRACTORA,I- PATRIMUL COMPUESTO Aer.Susp.x 200 DOSIS ...,1.0,1.0,19576.270000,19576.27,0,Completa,10305,Completada,LOC,INSTALACIONES TRC SAC,2018-07-04 18:28:46.013,Crédito,2018-07
2,1000000,20494,11006,I- PATRIMUL 20 MCG Aer.Susp.x 200 DOSIS (MEGA...,I- PATRIMUL COMPUESTO Aer.Susp.x 200 DOSIS ...,20.0,20.0,70.200000,1404.00,0,Completa,10808,Completada,Compra Medicinas/M.Medicos,ALFARO S.A.C.,2020-12-03 13:07:45.984,L/120 Días,2020-12
3,1000000,6,8010,ARCO EN C CIOS ALPHA,CORTIPREX 5MG/5ML SUSP. X 100 ML,1.0,1.0,164749.423730,164749.42,0,Completa,30307,Completada,LOC,SIEMENS HEALTHCARE S.A.C,2018-05-03 17:25:35.040,10% Adelantado - Saldo Crédito (12),2018-05
4,1000000,88,8010,CORTIPREX JBE,CORTIPREX 5MG/5ML SUSP. X 100 ML,2.0,2.0,15.439970,30.88,0,Completa,10304,Completada,LOC,ALFARO S.A.C.,2018-05-08 08:30:11.002,L/90 Días,2018-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44073,1000000,37290,7741,SULFADIAZINA 400 G,SULFADIAZINA 400 G,5.0,5.0,67.627119,338.14,0,Completa,10808,Completada,Compra Medicinas/M.Medicos,IMPROVENG S.R.L.,2024-03-19 14:17:40.963,L/120 Días,2024-03
44074,1000000,37498,7633,TROPICAMIDA 1% X 15 ML,TROPICAMIDA 1% X 15 ML,15.0,15.0,35.644068,534.66,0,Completa,10808,Aprobada,Compra Medicinas/M.Medicos,DIMEXA,2024-04-05 15:51:04.003,L/90 Días,2024-04
44075,1000000,37498,13580,FENILEFRINA 10% X 5 ML,FENILEFRINA 10% X 5 ML,15.0,15.0,40.347458,605.21,0,Completa,10808,Aprobada,Compra Medicinas/M.Medicos,DIMEXA,2024-04-05 15:51:04.003,L/90 Días,2024-04
44076,1000000,37499,13582,VABYSMO 120 MG,VABYSMO 120 MG,1.0,1.0,2719.161017,2719.16,0,Completa,10808,Completada,Compra Medicinas/M.Medicos,QUIMICA SUIZA S.A.,2024-04-05 16:06:08.006,Contado,2024-04


In [59]:
data_frame_2 = data_frame_1.drop(columns=['FechaPreparación'])

In [61]:
agrupacion = data_frame_2.groupby(list(data_frame_2.columns)).size().reset_index(name='cantidad_items')


In [64]:
agrupacion.count

<bound method DataFrame.count of        Companía  OrdenCompra  Item_Commodity  \
0       1000000            5           11006   
1       1000000            6            8010   
2       1000000           10            6638   
3       1000000           10            7291   
4       1000000           16            7110   
...         ...          ...             ...   
43781   1000000        37578            7885   
43782   1000000        37578           12580   
43783   1000000        37579            8859   
43784   1000000        37579            8860   
43785   1000000        37585            7975   

                                        Descripcion  \
0                                CAMPANA EXTRACTORA   
1                              ARCO EN C CIOS ALPHA   
2                                     SUPRADYN  TAB   
3                            PANADOL ANTIGRI.X2 TAB   
4                              SIMDAX(LEVOSIMENDAN)   
...                                             ...   
43781